In [1]:
import autograd
import autograd.numpy as np


# Paragami debugging

In [19]:
import paragami
import copy
import unittest
from numpy.testing import assert_array_almost_equal
import scipy as sp
import scipy as osp

import itertools
import json
import collections
import time

In [46]:
class DummyTest(unittest.TestCase):
    pass
testcase = DummyTest()
self = DummyTest()
check_equal = assert_array_almost_equal

# array_pattern = paragami.NumericArrayPattern(
#     shape=(4, ), lb=-1, ub=10.0)
# pattern = paragami.PatternArray((2, 3), array_pattern)

pattern = paragami.NumericArrayPattern(shape=(500, 10, 10), lb=-1, ub=10.0)

In [76]:
import cProfile
import pstats
valid_value = pattern.random()

# Execute required methods.
empty_val = pattern.empty(valid=True)
profile = True

if profile:
    pr = cProfile.Profile()
    pr.enable()

tic = time.time()
for _ in range(1000):
    pattern.flatten(empty_val, free=False, validate_value=False)
tic = time.time() - tic; print(tic)

if profile:
    pr.disable()

0.04148721694946289


In [74]:
ps = pstats.Stats(pr).strip_dirs().sort_stats('cumulative')
ps.print_stats()

         28152 function calls in 0.047 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5    0.000    0.000    0.047    0.009 interactiveshell.py:3281(run_code)
        5    0.000    0.000    0.047    0.009 {built-in method builtins.exec}
        1    0.002    0.002    0.047    0.047 <ipython-input-73-4629c68d8416>:14(<module>)
     1000    0.003    0.000    0.045    0.000 numeric_array_patterns.py:236(flatten)
     1000    0.026    0.000    0.026    0.000 {method 'flatten' of 'numpy.ndarray' objects}
     2000    0.002    0.000    0.014    0.000 tracer.py:35(f_wrapped)
     2000    0.002    0.000    0.009    0.000 <__array_function__ internals>:2(atleast_1d)
     1000    0.001    0.000    0.009    0.000 numeric_array_patterns.py:196(validate_folded)
     2000    0.001    0.000    0.007    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
     2000    0.003    0.000    0.005    0.000 sha

In [ ]:
empty_val = pattern.empty(valid=False)

random_val = pattern.random()
pattern.flatten(random_val, free=False)

str(pattern)

pattern.empty_bool(True)

# Make sure to test != using a custom test.
testcase.assertTrue(pattern == pattern)

###############################
# Test folding and unfolding.
for free in [True, False, None]:
    for free_default in [True, False, None]:
        pattern.free_default = free_default
        if (free_default is None) and (free is None):
            with testcase.assertRaises(ValueError):
                flat_val = pattern.flatten(valid_value, free=free)
            with testcase.assertRaises(ValueError):
                folded_val = pattern.fold(flat_val, free=free)
        else:
            flat_val = pattern.flatten(valid_value, free=free)
            testcase.assertEqual(len(flat_val), pattern.flat_length(free))
            folded_val = pattern.fold(flat_val, free=free)
            check_equal(valid_value, folded_val)
            if hasattr(valid_value, 'shape'):
                testcase.assertEqual(valid_value.shape, folded_val.shape)

